In [320]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder
from numpy import asarray

from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier

import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.pipeline import make_pipeline

In [321]:
fdf =pd.read_csv('five_sec')
fdf.head()

,dataset_5secondWindow.csv,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,0.0,16170.0,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,1.0,15871.0,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,2.0,16811.0,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,3.0,15831.0,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,4.0,876.0,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,U1


In [322]:
fdf.isnull().sum().sort_values(ascending=False)

activityrecognition#0                5894
android.sensor.step_counter#std      5662
android.sensor.proximity#std         5611
android.sensor.proximity#max         4775
android.sensor.proximity#min         4775
                                     ... 
android.sensor.accelerometer#min        1
android.sensor.accelerometer#mean       1
activityrecognition#1                   1
time                                    1
user                                    1
Length: 71, dtype: int64

In [323]:
fdf.shape

(5894, 71)

In [324]:
fdf = fdf.dropna(how='all')
fdf.shape

(5893, 71)

In [325]:
print(fdf['user'].isna().sum())
print(fdf['target'].isna().sum())

0
0


In [326]:
fdf.isnull().sum().sort_values(ascending=False).head(30)

activityrecognition#0                              5893
android.sensor.step_counter#std                    5661
android.sensor.proximity#std                       5610
android.sensor.proximity#max                       4774
android.sensor.proximity#min                       4774
android.sensor.proximity#mean                      4774
android.sensor.pressure#std                        4765
speed#std                                          4575
android.sensor.step_counter#min                    4406
android.sensor.step_counter#max                    4406
android.sensor.step_counter#mean                   4406
android.sensor.pressure#max                        3975
android.sensor.pressure#min                        3975
android.sensor.pressure#mean                       3975
android.sensor.light#std                           3769
speed#min                                          2373
speed#mean                                         2373
speed#max                                       

In [327]:
#Dropping the columns with more than 1000 Nans
fdf = fdf.drop(['dataset_5secondWindow.csv', 'id','activityrecognition#0','android.sensor.step_counter#std','android.sensor.proximity#std','android.sensor.proximity#max','android.sensor.proximity#min','android.sensor.proximity#mean','android.sensor.pressure#std','speed#std','android.sensor.step_counter#min','android.sensor.step_counter#max','android.sensor.step_counter#mean','android.sensor.pressure#max','android.sensor.pressure#min','android.sensor.pressure#mean','android.sensor.light#std','speed#min','speed#mean','speed#max','sound#std','android.sensor.light#max','android.sensor.light#mean','android.sensor.light#min','sound#max','sound#min','sound#mean','android.sensor.magnetic_field_uncalibrated#std','android.sensor.magnetic_field_uncalibrated#mean','android.sensor.magnetic_field_uncalibrated#mean','android.sensor.magnetic_field_uncalibrated#min','android.sensor.magnetic_field_uncalibrated#max','android.sensor.gyroscope_uncalibrated#max','android.sensor.gyroscope_uncalibrated#mean','android.sensor.game_rotation_vector#max','android.sensor.game_rotation_vector#min','android.sensor.game_rotation_vector#mean','android.sensor.gyroscope_uncalibrated#min','android.sensor.rotation_vector#std','android.sensor.rotation_vector#mean','android.sensor.rotation_vector#max','android.sensor.rotation_vector#min','android.sensor.game_rotation_vector#std','android.sensor.gyroscope_uncalibrated#std','android.sensor.gyroscope#std','android.sensor.gyroscope#max','android.sensor.gyroscope#min','android.sensor.gyroscope#mean'], axis=1)
#,'android.sensor.step_counter#std','android.sensor.proximity#std','android.sensor.proximity#max','android.sensor.proximity#min','android.sensor.proximity#mean','android.sensor.pressure#std','speed#std','android.sensor.step_counter#min','android.sensor.step_counter#max','android.sensor.step_counter#mean','android.sensor.pressure#max','android.sensor.pressure#min','android.sensor.pressure#mean','android.sensor.light#std','speed#min','speed#mean','speed#max','sound#std','android.sensor.light#max','android.sensor.light#mean','android.sensor.light#min',

In [328]:
print(fdf.isnull().sum().sort_values(ascending=False).head(30))
print(fdf.shape)

android.sensor.gravity#std                 537
android.sensor.linear_acceleration#std     523
android.sensor.orientation#std             520
android.sensor.magnetic_field#std          510
android.sensor.gravity#max                 497
android.sensor.orientation#max             497
android.sensor.orientation#min             497
android.sensor.gravity#mean                497
android.sensor.gravity#min                 497
android.sensor.orientation#mean            497
android.sensor.linear_acceleration#min     494
android.sensor.linear_acceleration#max     494
android.sensor.linear_acceleration#mean    494
android.sensor.magnetic_field#mean         493
android.sensor.magnetic_field#min          493
android.sensor.magnetic_field#max          493
target                                       0
time                                         0
activityrecognition#1                        0
android.sensor.accelerometer#std             0
android.sensor.accelerometer#max             0
android.senso

In [329]:
fdf = fdf.dropna()

In [330]:
fdf = fdf.sort_values(by='user')
fdf

,time,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,...,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,target,user
5892,55.0,100.0,11.051009,5.203285,19.921068,4.745098,9.806650,9.806649,9.806651,3.594931e-07,...,43.983419,42.936376,45.081787,1.073626,260.716040,144.046424,401.652939,128.622821,Walking,U1
2266,93.0,100.0,10.155912,10.106256,10.213363,0.033405,9.806650,9.806650,9.806650,1.785139e-07,...,42.086943,42.086943,42.086943,0.652390,227.683890,226.550681,228.587043,1.037490,Still,U1
4643,95.0,100.0,10.582088,8.589124,13.955928,1.944233,9.806650,9.806649,9.806650,3.083597e-07,...,55.479466,52.826711,57.889720,2.018881,165.375072,152.174761,172.963132,7.937969,Walking,U1
2271,86.0,100.0,10.195350,8.977111,11.067605,0.585297,9.806650,9.806650,9.806651,3.776391e-07,...,5.463656,4.092391,7.864070,1.112907,314.832106,314.137076,315.459098,0.583711,Train,U1
2272,39.0,100.0,9.911048,7.994811,13.290088,1.708189,9.806650,9.806650,9.806651,3.025809e-07,...,43.185007,42.295172,45.019001,1.242161,205.647571,176.402982,238.673703,22.308571,Walking,U1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3576,56.0,100.0,8.883861,4.312250,14.790456,3.825252,9.806650,9.806650,9.806651,1.147585e-06,...,47.911160,47.719436,48.145491,0.216200,353.268751,345.918404,364.295058,8.672417,Walking,U9
2175,70.0,8.0,10.439644,9.880899,10.974127,0.274404,9.806650,9.806650,9.806651,3.928756e-07,...,67.829159,60.850290,74.838080,5.025628,131.056961,126.437513,138.261912,4.788278,Train,U9
1596,130.0,8.0,10.648997,10.245279,10.925646,0.199106,9.806650,9.806650,9.806650,5.828058e-07,...,176.819096,170.859569,181.002676,4.081061,117.065591,103.584544,119.621123,4.914210,Train,U9
1705,122.0,8.0,10.557161,10.316969,10.756603,0.117200,9.806651,9.806651,9.806651,3.810355e-07,...,124.497102,121.104004,129.205717,3.041527,105.899895,61.181022,130.674852,33.495886,Train,U9


In [331]:
fdf['user'].value_counts()

U1     2470
U12     803
U10     517
U7      378
U3      330
U2      270
U4      206
U11     167
U13      84
U9       79
U5       13
Name: user, dtype: int64

In [332]:
fdf.reset_index(inplace=True)

In [333]:
fdf = fdf.drop(['index','user'], axis=1)

In [334]:
oe = OrdinalEncoder()
cd = asarray(fdf['target'])
fdf['target'] = oe.fit_transform(cd.reshape(-1,1))

In [335]:
X,y = fdf.drop(['target'], axis=1), fdf['target']

In [336]:
data_test = fdf.iloc[:2470, :]
data_train = fdf.iloc[2470: , :]

In [337]:
X_train = data_train.iloc[:,:22]
y_train = data_train.iloc[:, 22:]

X_test = data_test.iloc[:, :22]
y_test = data_test.iloc[:, 22:]

In [338]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
y_test

(2847, 22)
(2847, 1)
(2470, 22)
(2470, 1)


,target
0,4.0
1,2.0
2,4.0
3,3.0
4,4.0
...,...
2465,0.0
2466,2.0
2467,1.0
2468,2.0


In [339]:
tree_classifiers= {
  "Decision Tree": DecisionTreeClassifier(random_state=0),
  "Extra Trees":ExtraTreesClassifier(random_state=0),
  "Random Forest": RandomForestClassifier(random_state=0)
}

In [340]:
tree_classifiers = {name: make_pipeline(model) for name, model in tree_classifiers.items()}
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():
    
    start_time = time.time()
    model.fit(X_train, y_train)
    total_time = time.time() - start_time
        
    pred = model.predict(X_test)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_test, pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='Blues')

C:\Users\Abubakr\AppData\Local\Temp\ipykernel_1992\1374120683.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
c:\Users\Abubakr\Anaconda3\envs\ai\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Abubakr\AppData\Local\Temp\ipykernel_1992\1374120683.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
c:\Users\Abubakr\Anaconda3\envs\ai\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sampl

,Model,Accuracy,Bal Acc.,Time
1,Extra Trees,48.582996,51.304159,0.774389
2,Random Forest,48.421053,49.468480,1.046037
3,Decision Tree,35.870445,35.085195,0.146907
